In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [29]:
#her OP nin unique ismi var, bunlari kunlarak lines i dogru formata gecirebilirsin.
np.unique(lines.loc[:,"Abbreviation of the operating point"].to_numpy())

array(['AA', 'AAGB', 'AAT', ..., 'ZWIP', 'ZWOL', 'ZZ'], dtype=object)

In [3]:
lines = pd.read_csv("linie-mit-betriebspunkten.xls.csv")
G = nx.Graph()

#dogrudan linesla calismak hatali, cunku lines'da bazi operation pointler birden fazla defa karsimiza cikiyor, her karsimiza cikisinda
#graph'a node eklemek istemiyoruz. Once lines dataframe ini her OP nin bir rowda kayitli oldugu bir formata donusturursen bu comment'in
#altindaki kod sana graph'i yaratir.
for index, row in lines.iterrows():
    G.add_nodes_from([(index, row.to_dict())])
    
line_ids = np.unique(lines.loc[:,"LINIE"].to_numpy())
for line_id in line_ids:
    operation_points = lines[lines["LINIE"] == line_id].sort_values(by='Kilometry of the operating point')
    operation_points = operation_points.index.to_numpy()
    edges = np.concatenate((operation_points[:-1][:,np.newaxis],operation_points[1:][:,np.newaxis]), axis=1)
    G.add_edges_from(edges)